In [191]:
import numpy as np
import pandas as pd
Series = pd.Series
DataFrame = pd.DataFrame
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from itertools import product

In [2]:
doc = pd.read_csv("US_Accidents_March23.csv")

In [3]:
doc.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
doc.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [5]:
doc = doc.reindex(columns=["ID", "Start_Time", "City", "State", "Temperature(F)", "Visibility(mi)", "Wind_Speed(mph)", "Precipitation(in)", "Weather_Condition"])
doc.head(100)

,ID,Start_Time,City,State,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition
0,A-1,2016-02-08 05:46:00,Dayton,OH,36.9,10.0,NaN,0.02,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,OH,37.9,10.0,NaN,0.00,Light Rain
2,A-3,2016-02-08 06:49:27,Williamsburg,OH,36.0,10.0,3.5,NaN,Overcast
3,A-4,2016-02-08 07:23:34,Dayton,OH,35.1,9.0,4.6,NaN,Mostly Cloudy
4,A-5,2016-02-08 07:39:07,Dayton,OH,36.0,6.0,3.5,NaN,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...
95,A-96,2016-02-11 07:28:16,Pataskala,OH,7.0,10.0,NaN,NaN,Overcast
96,A-97,2016-02-11 07:29:06,Cleveland,OH,15.1,10.0,18.4,NaN,Mostly Cloudy
97,A-98,2016-02-11 07:53:55,Dayton,OH,9.0,9.0,NaN,NaN,Clear
98,A-99,2016-02-11 08:07:02,Dayton,OH,7.5,10.0,4.6,NaN,Scattered Clouds


In [6]:
doc = doc.dropna(subset=["Weather_Condition"])
len(doc)

7554935

In [7]:
cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "Austin", "Jacksonville"]

In [8]:
# For experimenting with a data set small enough that operations run
# instantly. Not for analysis: just data manipulation.
doc_head = doc.head(10000).copy()
doc_head = doc_head.reindex(columns=["Start_Time", "City", "Weather_Condition"])

In [9]:
doc = doc[doc["City"].isin(cities)]
doc.head()

,ID,Start_Time,City,State,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition
42866,A-42867,2016-06-21 10:46:30,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42867,A-42868,2016-06-21 10:49:21,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42881,A-42882,2016-06-21 10:51:45,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42883,A-42884,2016-06-21 10:56:24,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42886,A-42887,2016-06-21 10:57:39,San Diego,CA,69.1,9.0,8.1,NaN,Scattered Clouds


In [10]:
DataFrame(doc_head.groupby(["City", "Weather_Condition"])).iloc[0, 1]

,Start_Time,City,Weather_Condition
1465,2016-06-27 18:35:11,Acampo,Clear
1574,2016-06-28 16:45:10,Acampo,Clear
1967,2016-07-03 08:09:37,Acampo,Clear
2187,2016-07-05 17:09:47,Acampo,Clear
2652,2016-07-10 11:15:17,Acampo,Clear
2953,2016-07-13 18:51:38,Acampo,Clear
3054,2016-07-14 16:49:45,Acampo,Clear
4185,2016-07-25 19:53:25,Acampo,Clear
4314,2016-07-26 15:49:08,Acampo,Clear
6851,2016-12-12 15:33:24,Acampo,Clear


In [11]:
def previous_hour_mark(date_str):
    date, time = date_str.split()
    time = time[:2]
    return date + " " + time

In [12]:
doc["Start_Time"] = doc["Start_Time"].apply(previous_hour_mark)
doc

,ID,Start_Time,City,State,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition
42866,A-42867,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42867,A-42868,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42881,A-42882,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42883,A-42884,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42886,A-42887,2016-06-21 10,San Diego,CA,69.1,9.0,8.1,NaN,Scattered Clouds
...,...,...,...,...,...,...,...,...,...
7728364,A-7777732,2019-08-23 13,Los Angeles,CA,82.0,9.0,0.0,0.0,Fair
7728374,A-7777742,2019-08-23 15,San Diego,CA,74.0,10.0,7.0,0.0,Fair
7728376,A-7777744,2019-08-23 15,Los Angeles,CA,74.0,10.0,8.0,0.0,Fair
7728377,A-7777745,2019-08-23 16,San Diego,CA,75.0,10.0,7.0,0.0,Fair


In [13]:
def most_common(df, column="Weather_Condition"):
    return Series([df[column].mode()[0], len(df[column])]) 

In [14]:
aggregated = doc.groupby(["Start_Time", "City"]).apply(most_common)
aggregated

0  1
Start_Time    City                         
2016-03-22 19 Los Angeles          Clear  1
2016-03-22 20 Los Angeles          Clear  2
              San Diego            Clear  1
2016-03-22 22 Los Angeles          Clear  2
2016-03-23 00 San Diego    Partly Cloudy  1
...                                  ... ..
2023-03-31 21 Chicago             Cloudy  3
              Phoenix      Mostly Cloudy  1
2023-03-31 22 Chicago      Mostly Cloudy  1
              Dallas                Fair  2
2023-03-31 23 Dallas                Fair  1

[270364 rows x 2 columns]

In [15]:
ag_head = aggregated.head(10_000)
ag_head

0  1
Start_Time    City                         
2016-03-22 19 Los Angeles          Clear  1
2016-03-22 20 Los Angeles          Clear  2
              San Diego            Clear  1
2016-03-22 22 Los Angeles          Clear  2
2016-03-23 00 San Diego    Partly Cloudy  1
...                                  ... ..
2016-09-04 22 Los Angeles          Clear  2
2016-09-05 01 Los Angeles       Overcast  1
2016-09-05 02 Los Angeles       Overcast  3
2016-09-05 03 Los Angeles  Partly Cloudy  1
              San Antonio  Mostly Cloudy  1

[10000 rows x 2 columns]

In [16]:
ag_head.iloc[2]

0    Clear
1        1
Name: (2016-03-22 20, San Diego), dtype: object

In [17]:
len(ag_head)

10000

In [18]:
[ag_head.iloc[0][1], ag_head.iloc[0].name[1], ag_head.iloc[0].name[0], ag_head.iloc[0][0]]
data = {
    "Crashes": [ag_head.iloc[i][1] for i in range(len(ag_head))],
    "City": [ag_head.iloc[i].name[1] for i in range(len(ag_head))],
    "Hour": [ag_head.iloc[i].name[0][-2:] for i in range(len(ag_head))],
    "Weather": [ag_head.iloc[i][0] for i in range(len(ag_head))],
}
flat_head = DataFrame(data)
flat_head

,Crashes,City,Hour,Weather
0,1,Los Angeles,19,Clear
1,2,Los Angeles,20,Clear
2,1,San Diego,20,Clear
3,2,Los Angeles,22,Clear
4,1,San Diego,00,Partly Cloudy
...,...,...,...,...
9995,2,Los Angeles,22,Clear
9996,1,Los Angeles,01,Overcast
9997,3,Los Angeles,02,Overcast
9998,1,Los Angeles,03,Partly Cloudy


In [19]:
# There is surely a better way to do this
data = {
    "Crashes": [aggregated.iloc[i][1] for i in range(len(aggregated))],
    "City": [aggregated.iloc[i].name[1] for i in range(len(aggregated))],
    "Hour": [aggregated.iloc[i].name[0][-2:] for i in range(len(aggregated))],
    "Weather": [aggregated.iloc[i][0] for i in range(len(aggregated))],
}
flat = DataFrame(data)
flat

,Crashes,City,Hour,Weather
0,1,Los Angeles,19,Clear
1,2,Los Angeles,20,Clear
2,1,San Diego,20,Clear
3,2,Los Angeles,22,Clear
4,1,San Diego,00,Partly Cloudy
...,...,...,...,...
270359,3,Chicago,21,Cloudy
270360,1,Phoenix,21,Mostly Cloudy
270361,1,Chicago,22,Mostly Cloudy
270362,2,Dallas,22,Fair


In [73]:
def collapser(category, to_keep, other_str="Other"):
    s = set(to_keep)
    if category in s:
        return category
    else:
        return other_str

In [78]:
to_keep = ["Fair", "Mostly Cloudy", "Clear", "Cloudy", "Partly Cloudy", "Overcast", "Light Rain", "Scattered Clouds", "Haze", "Fog", "Rain", "Light Snow", "Light Drizzle", "Heavy Rain"]
flat["Weather"] = flat["Weather"].apply(lambda x : collapser(x, to_keep, other_str="Other"))
flat["Weather"].value_counts()

Weather
Fair                80024
Mostly Cloudy       44498
Clear               41429
Cloudy              29159
Partly Cloudy       25629
Overcast            18302
Light Rain           9088
Scattered Clouds     8128
Other                4805
Haze                 3205
Fog                  2173
Rain                 1260
Light Snow           1035
Light Drizzle         857
Heavy Rain            772
Name: count, dtype: int64

In [80]:
assert flat["Weather"].value_counts().sum() == 270364

In [84]:
train, test = train_test_split(flat.copy(), 
                               shuffle=True,
                               random_state=614,
                               test_size=.2)

In [41]:
train.sample(5)

,Crashes,City,Hour,Weather
105988,2,Los Angeles,07,Fair
62437,5,Austin,11,Overcast
39197,1,Houston,23,Partly Cloudy
89963,1,Dallas,19,Clear
55013,1,Houston,16,Overcast


In [22]:
# free up some memory here
del doc
del aggregated

In [148]:
def one_hot_encode(df, in_place=False):
    if not in_place:
        df = df.copy()
    df.loc[:, pd.get_dummies(df["City"]).columns] = pd.get_dummies(df["City"], dtype=int).copy()
    del df["Austin"]
    del df["City"]
    df.loc[:, pd.get_dummies(df["Weather"]).columns] = pd.get_dummies(df["Weather"], dtype=int).copy()
    del df["Other"]
    del df["Weather"]
    return df

In [122]:
def circular_encode_hour(df, in_place=False):
    if not in_place:
        df = df.copy()
    df["HourSin"] = df["Hour"]
    df["HourCos"] = df["Hour"]
    df["HourSin"] = df["HourSin"].apply(lambda x : np.sin(2*np.pi*int(x) / 24))
    df["HourCos"] = df["HourCos"].apply(lambda x : np.cos(2*np.pi*int(x) / 24))
    del df["Hour"]
    return df

In [111]:
n

,Crashes,City,Hour,Weather,Chicago,Dallas,Houston,Jacksonville,Los Angeles,New York,...,Haze,Light Drizzle,Light Rain,Light Snow,Mostly Cloudy,Other,Overcast,Partly Cloudy,Rain,Scattered Clouds
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165703,1,Dallas,02,Cloudy,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41012,3,Chicago,14,Mostly Cloudy,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
61434,8,Austin,08,Overcast,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
242204,6,Dallas,06,Fair,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
n = one_hot_encode(train)
n["HourSin"] = n["Hour"]
n["HourCos"] = n["Hour"]
n["HourSin"] = n["HourSin"].apply(lambda x : np.sin(2*np.pi*int(x) / 24))
n["HourCos"] = n["HourCos"].apply(lambda x : np.cos(2*np.pi*int(x) / 24))
n

,Crashes,Hour,Chicago,Dallas,Houston,Jacksonville,Los Angeles,New York,Philadelphia,Phoenix,...,Light Rain,Light Snow,Mostly Cloudy,Other,Overcast,Partly Cloudy,Rain,Scattered Clouds,HourSin,HourCos
60103,3,18,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,-1.000000,-1.836970e-16
165703,1,02,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.500000,8.660254e-01
249617,5,07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.965926,-2.588190e-01
41012,3,14,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,-0.500000,-8.660254e-01
117208,1,16,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,-0.866025,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,18,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,-1.000000,-1.836970e-16
61434,8,08,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.866025,-5.000000e-01
242204,6,06,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
71451,1,15,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.707107,-7.071068e-01


In [147]:
circular_encode_hour(one_hot_encode(train)).iloc[:, 1:].values

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -1.00000000e+00, -1.83697020e-16],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  5.00000000e-01,  8.66025404e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  9.65925826e-01, -2.58819045e-01],
       ...,
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  6.12323400e-17],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.07106781e-01, -7.07106781e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -9.65925826e-01,  2.58819045e-01]])

In [44]:
train.loc[:, pd.get_dummies(train["City"]).columns] = pd.get_dummies(train['City'], dtype=int).copy()
train

,Crashes,City,Hour,Weather,Austin,Chicago,Dallas,Houston,Jacksonville,Los Angeles,New York,Philadelphia,Phoenix,San Antonio,San Diego
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,0,0,1,0,0
165703,1,Dallas,02,Cloudy,0,0,1,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,0,0,0,0,1
41012,3,Chicago,14,Mostly Cloudy,0,1,0,0,0,0,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,0,0,0,1,0
61434,8,Austin,08,Overcast,1,0,0,0,0,0,0,0,0,0,0
242204,6,Dallas,06,Fair,0,0,1,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,0,1,0,0,0,0,0,0,0,0


In [45]:
del train["Austin"]
train

,Crashes,City,Hour,Weather,Chicago,Dallas,Houston,Jacksonville,Los Angeles,New York,Philadelphia,Phoenix,San Antonio,San Diego
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,0,1,0,0
165703,1,Dallas,02,Cloudy,0,1,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,0,0,0,1
41012,3,Chicago,14,Mostly Cloudy,1,0,0,0,0,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,0,0,1,0
61434,8,Austin,08,Overcast,0,0,0,0,0,0,0,0,0,0
242204,6,Dallas,06,Fair,0,1,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,1,0,0,0,0,0,0,0,0


In [24]:
train.loc[:, pd.get_dummies(train["Weather"]).columns] = pd.get_dummies(train['Weather'], dtype=int).copy()
train

,Crashes,City,Hour,Weather,Austin,Chicago,Dallas,Houston,Jacksonville,Los Angeles,...,Thunder,Thunder / Windy,Thunder / Wintry Mix,Thunder in the Vicinity,Thunderstorm,Thunderstorms and Rain,Volcanic Ash,Widespread Dust,Wintry Mix,Wintry Mix / Windy
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165703,1,Dallas,02,Cloudy,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41012,3,Chicago,14,Mostly Cloudy,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61434,8,Austin,08,Overcast,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242204,6,Dallas,06,Fair,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
train.loc[:, pd.get_dummies(train["Hour"]).columns] = pd.get_dummies(train['Hour'], dtype=int).copy()
train

,Crashes,City,Hour,Weather,Austin,Chicago,Dallas,Houston,Jacksonville,Los Angeles,...,14,15,16,17,18,19,20,21,22,23
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
165703,1,Dallas,02,Cloudy,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41012,3,Chicago,14,Mostly Cloudy,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
61434,8,Austin,08,Overcast,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242204,6,Dallas,06,Fair,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [26]:
# With no interaction terms
no_interaction_lr = LinearRegression(copy_X = True, positive=True, fit_intercept=False)

# Build the columns we use since there are many.

categories = list(train["Hour"].unique()) + list(train["Weather"].unique()) + list(train["City"].unique())

no_interaction_lr.fit(train[categories].values,
                      train['Crashes'].values)

LinearRegression(fit_intercept=False, positive=True)

In [27]:
assert len(categories) == 111

In [28]:
no_interaction_lr.coef_

array([1.40884151e+00, 9.45262351e-02, 2.05534481e+00, 1.30885459e+00,
       1.80931529e+00, 4.61812675e-01, 1.12700940e+00, 5.85259779e-01,
       1.61731790e+00, 1.11730131e+00, 1.04783131e+00, 8.50129540e-01,
       2.30495462e-01, 1.34177742e+00, 1.57393241e-01, 1.89229197e-01,
       1.93125511e+00, 1.37344051e+00, 2.30459040e+00, 1.27992113e+00,
       2.61416194e-01, 0.00000000e+00, 1.85092247e-01, 1.33404310e-01,
       2.49294721e-01, 5.20358911e-01, 7.42167745e-01, 3.46876579e-01,
       1.84178839e-01, 3.49563595e-01, 3.40760213e-01, 7.85154951e-01,
       1.13079005e+00, 2.24514200e-01, 8.52741464e-02, 1.58115868e-01,
       1.18584571e+00, 7.86181510e-01, 7.04760497e-01, 9.11389841e-01,
       1.83256227e+00, 3.03007425e-01, 9.51334454e-01, 1.77759638e+00,
       7.45898031e-01, 6.22989893e-01, 2.92944106e+00, 1.13048381e+00,
       2.27309180e+00, 3.93217359e-01, 0.00000000e+00, 2.24927672e+00,
       0.00000000e+00, 1.11334694e+00, 9.57882368e-01, 3.67165600e-01,
      

In [29]:
no_interaction_lr.predict(train[categories].values[:10])

array([2.26094268, 2.54879067, 3.48462453, 1.82112646, 4.3337926 ,
       1.07345495, 3.63236053, 3.80308267, 4.1510988 , 4.05514073])

In [30]:
no_interaction_lr.score(train[categories].values,
                      train['Crashes'].values)

0.1473255084360212

In [31]:
def encode(city, hour, weather):
    l = list(train[categories].columns)
    city_index = l.index(city)
    try:
        hour_index = l.index(hour)
    except ValueError:
        s = str(hour)
        if len(s) == 1:
            s = "0" + s
        hour_index = l.index(s)
    weather_index = l.index(weather)
    ans = [0 for _ in range(111)]
    ans[city_index] = 1
    ans[hour_index] = 1
    ans[weather_index] = 1
    return np.array(ans)
    

In [32]:
no_interaction_lr.predict([encode("Austin", "06", "Clear")])

array([3.21342184])

In [33]:
no_interaction_lr.predict([encode("Austin", 0, "Clear")])

array([2.02903766])

In [135]:
flat.sort_values(by=["City", "Hour"])

,Crashes,City,Hour,Weather
3287,1,Austin,00,Partly Cloudy
4289,1,Austin,00,Mostly Cloudy
13959,5,Austin,00,Light Rain
16877,1,Austin,00,Partly Cloudy
21671,1,Austin,00,Overcast
...,...,...,...,...
269173,2,San Diego,23,Cloudy
269472,2,San Diego,23,Fog
269928,1,San Diego,23,Fair
270160,1,San Diego,23,Fair


In [164]:
grouped = flat.groupby(["City", "Hour"])
grouped_crashes = grouped["Crashes"]
#DataFrame(grouped_crashes)
grouped_crashes_mean  = grouped_crashes.agg("mean")
grouped_crashes_mean["Austin", "00"]

2.363013698630137

In [158]:
flat.iloc[0]

Crashes              1
City       Los Angeles
Hour                19
Weather          Clear
Name: 0, dtype: object

In [171]:
grouped_crashes_mean["Phoenix", "18"]

2.5251524390243905

In [167]:
Series(grouped_crashes_mean[train.iloc[i]["City"], train.iloc[i]["Hour"]] for i in range(len(train)))

0         2.525152
1         2.313333
2         2.663752
3         1.550600
4         4.484977
            ...   
216286    2.037149
216287    4.718732
216288    4.499743
216289    3.647314
216290    3.498411
Length: 216291, dtype: float64

In [35]:
flat.to_csv("flat.csv")

In [204]:
encoded_cities = ["Chicago", "Dallas", "Houston", "Jacksonville", "Los Angeles", "New York", "Philadelphia", "San Antonio", "Phoenix"]
encoded_weather = ["Fair", "Mostly Cloudy", "Clear", "Cloudy", "Partly Cloudy", "Overcast", "Light Rain", "Scattered Clouds", "Haze", "Fog", "Rain", "Light Snow", "Light Drizzle", "Heavy Rain"]
times = ["HourCos", "HourSin"]
cities_weather_prods = list(product(encoded_cities, encoded_weather))
cities_times_prods = list(product(encoded_cities, times))
weather_times_product = list(product(encoded_weather, times))
triple_product = list(product(encoded_cities, encoded_weather, times))

In [211]:
train_categories = circular_encode_hour(one_hot_encode(train))
train_categories

,Crashes,Chicago,Dallas,Houston,Jacksonville,Los Angeles,New York,Philadelphia,Phoenix,San Antonio,...,Light Drizzle,Light Rain,Light Snow,Mostly Cloudy,Overcast,Partly Cloudy,Rain,Scattered Clouds,HourSin,HourCos
60103,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,-1.000000,-1.836970e-16
165703,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.500000,8.660254e-01
249617,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.965926,-2.588190e-01
41012,3,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,-0.500000,-8.660254e-01
117208,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.866025,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,-1.000000,-1.836970e-16
61434,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.866025,-5.000000e-01
242204,6,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,6.123234e-17
71451,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.707107,-7.071068e-01


In [ ]:
for city, weather in cities_weather_prods:
    #print(city, weather)
    train_categories[f"{city} {weather}"] = train_categories[city]*train_categories[weather]
for city, time in cities_times_prods:
    train_categories[f"{city} {time}"] = train_categories[city]*train_categories[time]
for weather, time in weather_times_product:
    train_categories[f"{weather} {time}"] = train_categories[weather]*train_categories[time]
train_categories = train_categories.copy()
train_categories

In [210]:
import warnings


encoded_cities = ["Chicago", "Dallas", "Houston", "Jacksonville", "Los Angeles", "New York", "Philadelphia", "San Antonio", "Phoenix"]
encoded_weather = ["Fair", "Mostly Cloudy", "Clear", "Cloudy", "Partly Cloudy", "Overcast", "Light Rain", "Scattered Clouds", "Haze", "Fog", "Rain", "Light Snow", "Light Drizzle", "Heavy Rain"]
times = ["HourCos", "HourSin"]
cities_weather_prods = list(product(encoded_cities, encoded_weather))
cities_times_prods = list(product(encoded_cities, times))
weather_times_product = list(product(encoded_weather, times))
triple_product = list(product(encoded_cities, encoded_weather, times))
def add_in_twofold_interaction_terms(df):
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    df = df.copy()
    for city, weather in cities_weather_prods:
        df[f"{city} {weather}"] = df[city]*df[weather]
    for city, time in cities_times_prods:
        df[f"{city} {time}"] = df[city]*df[time]
    for weather, time in weather_times_product:
        df[f"{weather} {time}"] = df[weather]*df[time]
    df = df.copy()
    warnings.simplefilter(action='default', category=pd.errors.PerformanceWarning)
    return df


In [214]:
add_in_twofold_interaction_terms(circular_encode_hour(one_hot_encode(train)))

,Crashes,Chicago,Dallas,Houston,Jacksonville,Los Angeles,New York,Philadelphia,Phoenix,San Antonio,...,Fog HourCos,Fog HourSin,Rain HourCos,Rain HourSin,Light Snow HourCos,Light Snow HourSin,Light Drizzle HourCos,Light Drizzle HourSin,Heavy Rain HourCos,Heavy Rain HourSin
60103,3,0,0,0,0,0,0,0,1,0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
165703,1,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
249617,5,0,0,0,0,0,0,0,0,0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0
41012,3,1,0,0,0,0,0,0,0,0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
117208,1,0,0,0,0,1,0,0,0,0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,0,0,0,0,0,0,0,0,1,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
61434,8,0,0,0,0,0,0,0,0,0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0
242204,6,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71451,1,0,1,0,0,0,0,0,0,0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [221]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse

from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

In [131]:
kfold = KFold(n_splits=5, shuffle=True)

In [ ]:
mses = np.zeros((7,5))



for i, (train_index, test_index) in enumerate(kfold.split(train)):
    train_cv = train.iloc[train_index]
    test_cv = train.iloc[test_index]

    
    # baseline model
    grouped = train_cv.groupby(["City", "Hour"])
    grouped_crashes = grouped["Crashes"]
    grouped_crashes_mean = grouped_crashes.agg("mean")
    baseline = Series(grouped_crashes_mean[test_cv.iloc[i]["City"], test_cv.iloc[i]["Hour"]] for i in range(len(test_cv)))
    
    # Setting up the data without interaction terms
    one_hot = one_hot_encode(train_cv)
    circular_one_hot = circular_encode_hour(one_hot)
    no_interaction_args = (circular_one_hot.iloc[:, 1:].values, train_cv["Crashes"].values)
    no_int_test = circular_encode_hour(one_hot_encode(test_cv)).iloc[:, 1:].values
    
    # Linear regression without interaction
    no_interaction_lr = LinearRegression(copy_X=True)
    no_interaction_lr.fit(*no_interaction_args)
    no_int_pred = no_interaction_lr.predict(no_int_test)
    
    # Ridge regression without interaction
    no_int_rr = Ridge(copy_X=True)
    no_int_rr.fit(*no_interaction_args)
    no_int_rr_pred = no_int_rr.predict(no_int_test)
    
    # Elastic net without interaction
    no_int_en = ElasticNet(copy_X=True)
    no_int_en.fit(*no_interaction_args)
    no_int_en_pred = no_int_en.predict(no_int_test)
    
    # Interaction setup
    interaction_terms = add_in_twofold_interaction_terms(circular_one_hot)
    interaction_args = (interaction_terms.iloc[:, 1:].values, train_cv["Crashes"].values)
    int_test = add_in_twofold_interaction_terms(circular_encode_hour(one_hot_encode(test_cv))).iloc[:, 1:].values
    
    # Linear regression with twofold interaction
    twofold_interaction_lr = LinearRegression(copy_X=True, positive=True)
    twofold_interaction_lr.fit(*interaction_args)
    twofold_int_pred = twofold_interaction_lr.predict(int_test)
    
    # Ridge regression without interaction
    int_rr = Ridge(copy_X=True)
    int_rr.fit(*interaction_args)
    int_rr_pred = int_rr.predict(int_test)
    
    # Elastic net without interaction
    int_en = ElasticNet(copy_X=True)
    int_en.fit(*interaction_args)
    int_en_pred = int_en.predict(int_test)
    
    mses[0,i] = mse(test_cv["Crashes"].values, baseline)
    mses[1,i] = mse(test_cv["Crashes"].values, no_int_pred)
    mses[2,i] = mse(test_cv["Crashes"].values, no_int_rr_pred)
    mses[3,i] = mse(test_cv["Crashes"].values, no_int_en_pred)
    mses[4,i] = mse(test_cv["Crashes"].values, twofold_int_pred)
    mses[5,i] = mse(test_cv["Crashes"].values, int_rr_pred)
    mses[6,i] = mse(test_cv["Crashes"].values, int_en_pred)
mses

In [234]:
for i in range(7):
    print(np.round(np.mean(mses, axis=1)[i],4))


7.1027
7.5722
7.5722
8.5928
9.149284222059936e+19
7.3249
8.5928
